In [55]:
from bs4 import BeautifulSoup
import datetime
import numpy as np
import pandas as pd
import re
import requests
import codecs
import os

wdir = "C:/Users/MarekSedlacek/Downloads/W251_Project/SilkRoad2/"

In [213]:
#ACTUAL CODE FOR PULLING PRODUCT DATA

#Ensure the current directory is correctly set
os.chdir(wdir)
#Build a Pandas data frame to hold the new data
col_names = ["Title","Category","Price","Price Dollar","Vendor","Ships From", "Ships To", "Date","File"]
df = pd.DataFrame(columns = col_names)
df_vend = pd.DataFrame(columns = col_names)

c = 0 #Counter
non_decimal = re.compile(r'[^\d.]+')

#Loops through the directories, corresponding to Dates
for X in next(os.walk(os.getcwd()))[1]:
    #Sets the working directory to the item folder for the current date
    curr_date = datetime.datetime(*[int(item) for item in X.split('-')])
    
    #Explore the Items listing first
    os.chdir(os.getcwd()+"\\"+X+"\items")
    print os.getcwd()
    
    #Loops through all files in the Items directory
    for i in os.listdir(os.getcwd()): 
        #Adds the .html file extension to all files
        pre, ext = os.path.splitext(i)
        new_name = pre+".html"
        os.rename(os.path.join(os.getcwd(), i), os.path.join(os.getcwd(), new_name))
        
        try:
            if i.endswith(".html"): #Only reads HTML files
                c+=1 #Increments counter
                
                soup = BeautifulSoup(open(i,'r').read()) #Reads in the HTML file
                
                sidebar = soup.find("div",attrs={"class":"sidebar"}) #Sidebar section of the HTML
                Category = sidebar.find("a").get_text() #Pulls Category from the Sidebar

                body = soup.find("div",attrs={"class":"body"} )#Body section of the HTML
                title = body.find("h2").get_text() #Pulls product title
                price = float(non_decimal.sub("",body.find("div",attrs={"class":"price_big"}).get_text())) #Pulls product price
                price_dollar = price*658.79 #Converts bitcoin price to Dollars (price from 2014-06-02)
                vendor = body.find("a").get_text() #Pulls Vendor
                
                #Identifies Shipping To and From out of a block of text
                S1 = body.find_all("p")
                if S1[0].get_text()=="":
                    S1 = S1[1].get_text().replace("ships from: ","").replace("ships to: ","")
                else: S1 = S1[0].get_text().replace("ships from: ","").replace("ships to: ","")
                Ship_From = S1.splitlines()[1].strip()
                Ship_To = S1.splitlines()[2].strip()
                
                #Creates a numpy array with the scraped values
                new = np.array([title,Category,price,price_dollar,vendor,Ship_From,Ship_To,curr_date,i])
                #Appends array to data frame
                df = df.append(pd.DataFrame(new, index=col_names).transpose())
                
                #Dictionary, for when we migrate to Cassandra
                row = {"Date":curr_date,"Title":title,"Category":Category,"Price":price,"price_dollar":price_dollar,"Vendor":vendor,"Ships_From":Ship_From,"Ships_To":Ship_To}

                #if c%100==0: print c #Tracking metric

            else:
                continue

        except:
            print "File " + i + " has failed"
            
    os.chdir(wdir) #Reset current working directory
    
    #Explore the Users listings next
    os.chdir(os.getcwd()+"\\"+X+"\users")
    print os.getcwd()
    
    #Loops through all User folders
    for x in next(os.walk(os.getcwd()))[1]:
        os.chdir(os.getcwd()+"\\"+x)
        
        #Loops through all files in a User's directory (typically just one)
        for filename in os.listdir(os.getcwd()):
            #Adds the .html file extension to all files
            pre, ext = os.path.splitext(filename)
            new_name = pre+".html"
            os.rename(os.path.join(os.getcwd(), filename), os.path.join(os.getcwd(), new_name))

        soup = BeautifulSoup(open("items.html",'r').read()) #Reads in the HTML file
        body = soup.find("div",attrs={"class":"body"} ) #Body section of the HTML
        items = body.find_all("div",attrs={"class":"item"}) #Item section of the body

        #Loops through all item listings in the HTML body
        for i in range(0,len(items)):
            try: 
                titles = items[i].find("div",attrs={"class":"item_title"}).find("a").get_text() #Get Title
                vendor = items[i].find("div",attrs={"class":"item_details"}).find("a").get_text() #Get Vendor
                price = float(non_decimal.sub("",items[i].find("div",attrs={"class":"price_big"}).get_text())) #Get Price
                price_dollar = price*658.79 #Converts bitcoin price to Dollars (price from 2014-06-02)
                Ship_From = items[i].find("div",attrs={"class":"item_details"}).get_text().splitlines()[2].replace("ships from: ","").strip()
                Ship_To = items[i].find("div",attrs={"class":"item_details"}).get_text().splitlines()[3].replace("ships to: ","").strip()
                Category = "" #No Categories in this data

                #Creates a numpy array with the scraped values
                new = np.array([titles,Category,price,price_dollar,vendor,Ship_From,Ship_To,curr_date,x])
                #Appends array to data frame
                df_vend = df_vend.append(pd.DataFrame(new, index=col_names).transpose())

            except: continue
       
        os.chdir(wdir+"\\2014-06-02\\users") #Reset to User directory
            
    os.chdir(wdir) #Reset current working directory
        

C:\Users\MarekSedlacek\Downloads\W251_Project\SilkRoad2\2014-06-02\items
File raise-your-flag-sr-branded-flags-look-great-when-flying-from-your-totem%3Fvendor_feedback_page=3.html has failed
File rolex-daydate-president-full-yellow-gold-aaa%3Fvendor_feedback_page=26.html has failed
File victoria-s-secret-bikini-replica-leopard-black-white-vs7-15.html has failed
File yeah-we-ran-out-of-drugs-how-about-some-stickers-3-5-x-6-sr-in-white%3Fvendor_feedback_page=48.html has failed
File yeah-we-ran-out-of-drugs-how-about-some-stickers-4-x3-warhol-snowden%3Fvendor_feedback_page=48.html has failed
C:\Users\MarekSedlacek\Downloads\W251_Project\SilkRoad2\2014-06-02\users
C:\Users\MarekSedlacek\Downloads\W251_Project\SilkRoad2\2014-06-11\items


WindowsError: [Error 2] The system cannot find the file specified: 'C:\\Users\\MarekSedlacek\\Downloads\\W251_Project\\SilkRoad2\\2014-06-11\\users'

In [220]:
#Remove duplicates from "Feedback" pages
df = df.drop_duplicates(subset=['Vendor',"Title","Date"],keep="first")

#Re-indexes the dataframe
df=df.sort_values(by=["Vendor","Title"])
df.index = range(df.shape[0])

In [221]:
#Remove duplicates from "Feedback" pages
df_vend = df_vend.drop_duplicates(subset=['Vendor',"Title","Date"],keep="first")

#Re-indexes the dataframe
df_vend=df_vend.sort_values(by=["Vendor","Title"])
df_vend.index = range(df_vend.shape[0])

In [222]:
#Append the Items and Users data sets and removing duplicates
#Keeps Items row if available, because it has Category
df2 = df.append(df_vend)
df2 = df2.drop_duplicates(subset=['Vendor',"Title","Date"],keep="first")
df2=df2.sort_values(by=["Vendor","Title","Date"])
df2.index = range(df2.shape[0])
df2

,Title,Category,Price,Price Dollar,Vendor,Ships From,Ships To,Date,File
0,BUY BITCOINS WITH MONEYGRAM OR WESTERN UNION,,0,0,*REALDEAL*,United States,United States,2014-06-02 00:00:00,realdeal-df64e04e-08e3-4e36-abba-dd2656475052
1,BUY BITCOINS WITH NETSPEND RELOAD,Stimulants,0,0,*REALDEAL*,United States,United States,2014-06-02 00:00:00,buy-bitcoins-with-netspend-reload-realdeal.html
2,REAL GUCCI LADIES WATCH WORN TWICE,,0.754067,496.772,*REALDEAL*,United States,United States,2014-06-02 00:00:00,realdeal-df64e04e-08e3-4e36-abba-dd2656475052
3,10x 18-20mg 2C-B HCl caps!,2C-Family,0.110579,72.8483,03welle,Germany,Worldwide,2014-06-11 00:00:00,10x-18-20mg-2c-b-hcl-caps.html
4,1x 18-20mg 2C-B HCl caps!,2C-Family,0.013475,8.8772,03welle,Germany,Worldwide,2014-06-11 00:00:00,1x-18-20mg-2c-b-hcl-caps.html
5,5x 18-20mg 2C-B HCl caps!,2C-Family,0.059297,39.0643,03welle,Germany,Worldwide,2014-06-11 00:00:00,5x-18-20mg-2c-b-hcl-caps.html
6,1.1 GRAM LISTING,,0.140821,92.7715,1_fly_up,United States,United States,2014-06-02 00:00:00,1_fly_up
7,3.5 'Future Star Listings' 3.5,,0.352054,231.93,1_fly_up,United States,United States,2014-06-02 00:00:00,1_fly_up
8,MARCH PHARMA DANABOL (DBOL) 100tabs 10mg - AU...,,0.360828,237.71,1dboladay,Australia,Australia,2014-06-02 00:00:00,1dboladay
9,MARCH PHARMA DANABOL (DBOL) 500tabs 10mg - AU...,,1.2301,810.376,1dboladay,Australia,Australia,2014-06-02 00:00:00,1dboladay
